In [1]:
from __future__ import print_function
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
## Disable tf future deprecated messages
import logging
logging.getLogger('tensorflow').disabled = True
## Disable tf CUDA messages
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

%matplotlib inline

import sys
import random
import numpy as np
import pandas as pd

import utils
import correction
from models.dense import *
from models.gain import gain
from models.soft_impute import SoftImpute
from models.sinkhorn import OTimputer
from models.mida import mida

import scipy.stats

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer, SimpleImputer

import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
dtype = torch.cuda.FloatTensor

## Repeat all experiments for 10 runs

In [2]:
n_runs = 10

## Load data

In [3]:
data_missing, missing_mask, y = utils.load_miocardial()
## Inject noise
data_missing = utils.inject_noise(data_missing.copy(), noise_rate=.10)
## Replace missing values locations by 0
data_missing = data_missing * missing_mask

Dataset shape: (1700, 110)
5.62% missing data
Class distribution: (array([0, 1]), array([1429,  271]))


## Run OUR METHOD

In [4]:
params = {
    'nb_batches': 10,
    'reg_noise_std': .03,
    'net_input': 'data_corrupted',
    'net_params': {
        'input_channels':1,
        'output_channels':1,
        'channels_skip':4,
        'down_layers':[(24, 7, 2),
                       (46, 9, 1),
                       (96, 9, 5),
                       (96, 11, 1)],
        'need_sigmoid':True,
        'need_bias':True,
        'pad':'zero',
        'downsample_mode':'stride',
        'upsample_mode':'nearest',
        'act_fun':'LeakyReLU',
        'need1x1_up':True
    },# or a list containing layers sizes
    'adam_lr': .0001,
    'adam_weight_decay': 0.,
}

ours_accs, ours_aucs = [], []
for i in range(n_runs):
    ours_correction = correction.run(data_missing, 501, params, y=y, missing_mask=missing_mask, seed=i)
    ## Since our method returns 2 imputations select the best one
    scores_raw = utils.get_scores(ours_correction['raw_out'], y)
    acc = scores_raw['test_balanced_accuracy']
    auc = scores_raw['test_roc_auc_ovo']
    acc = acc.mean()
    acc_std = acc.std()
    auc = auc.mean()
    auc_std = auc.std()
    if 'masked_out' in ours_correction:
        scores_masked = utils.get_scores(ours_correction['masked_out'], y)
        acc_masked = scores_masked['test_balanced_accuracy']
        auc_masked = scores_masked['test_roc_auc_ovo']
        if auc_masked.mean() > auc:
            acc = acc_masked.mean()
            acc_std = acc_masked.std()
            auc = auc_masked.mean()
            auc_std = auc_masked.std()
    print(f'OUR METHOD RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%')
    ours_accs.append(acc.mean())
    ours_aucs.append(auc.mean())
print(f'\nOUR METHOD GLOBAL - acc: {round(np.array(ours_accs).mean() * 100, 2)}% +- {round(np.array(ours_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(ours_aucs).mean() * 100, 2)}% +- {round(np.array(ours_aucs).std() * 100, 2)}%')

Ite 00000 - 6.20 sec - Loss 0.224128 - ACC 74.52% - ACC Mean 74.52% - AUC 79.33% - AUC Mean 79.33% - Deter 000
Ite 00050 - 0.91 sec - Loss 0.108819 - ACC 74.79% - ACC Mean 74.22% - AUC 80.62% - AUC Mean 80.15% - Deter 042
Ite 00058 - 1.17 sec - Loss 0.102832 - ACC 75.25% - ACC Mean 74.14% - AUC 80.26% - AUC Mean 80.05% - Deter 050
Early stop ite 58, rollback to correction of ite 8, whith acc of 74.25% and auc of 81.87%
OUR METHOD RUN 1/10 - acc: 74.25% +- 0.0% - auc: 81.87% +- 0.0%
Ite 00000 - 0.89 sec - Loss 0.231244 - ACC 73.50% - ACC Mean 73.50% - AUC 81.59% - AUC Mean 81.59% - Deter 000
Ite 00050 - 0.86 sec - Loss 0.121219 - ACC 74.12% - ACC Mean 75.10% - AUC 79.24% - AUC Mean 80.50% - Deter 035
Ite 00065 - 1.00 sec - Loss 0.109825 - ACC 73.22% - ACC Mean 74.68% - AUC 77.77% - AUC Mean 80.21% - Deter 050
Early stop ite 65, rollback to correction of ite 15, whith acc of 75.89% and auc of 82.3%
OUR METHOD RUN 2/10 - acc: 75.89% +- 0.0% - auc: 82.3% +- 0.0%
Ite 00000 - 0.82 sec - Loss

## Run Median, Mean and KNN

In [5]:
data_missing_nans = np.where(missing_mask, data_missing, np.nan)

median_imputer = SimpleImputer(missing_values=np.nan, strategy='median')
mean_imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
knn_imputer = KNNImputer(n_neighbors=10, weights='uniform')

median_completed = median_imputer.fit_transform(data_missing_nans)
mean_completed = mean_imputer.fit_transform(data_missing_nans)
knn_completed = knn_imputer.fit_transform(data_missing_nans)

## All runs would be the same since deterministic methods
scores = utils.get_scores(median_completed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MEDIAN - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
median_accs = acc
median_aucs = auc

scores = utils.get_scores(mean_completed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MEAN - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
mean_accs = acc
mean_aucs = auc

scores = utils.get_scores(knn_completed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'KNN - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
knn_accs = acc
knn_aucs = auc

MEDIAN - acc: 62.42% +- 0.0% - auc: 68.86% +- 0.0%
MEAN - acc: 75.14% +- 0.0% - auc: 81.97% +- 0.0%
KNN - acc: 67.49% +- 0.0% - auc: 72.42% +- 0.0%


## Run GAIN

In [6]:
gain_parameters = {'batch_size': 128,
                   'hint_rate': .9,
                   'alpha': 100,
                   'iterations': 5000}

gain_accs, gain_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputed = gain(data_missing_nans, gain_parameters)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nGAIN RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    gain_accs.append(acc.mean())
    gain_aucs.append(auc.mean())
print(f'GAIN GLOBAL - acc: {round(np.array(gain_accs).mean() * 100, 2)}% +- {round(np.array(gain_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(gain_aucs).mean() * 100, 2)}% +- {round(np.array(gain_aucs).std() * 100, 2)}%')

100%|██████████| 5000/5000 [00:56<00:00, 88.59it/s]



GAIN RUN 1/10 - acc: 73.39% +- 2.98% - auc: 79.13% +- 1.98%



100%|██████████| 5000/5000 [00:59<00:00, 84.01it/s]



GAIN RUN 2/10 - acc: 72.01% +- 3.61% - auc: 78.77% +- 4.17%



100%|██████████| 5000/5000 [01:05<00:00, 76.71it/s]


...


GAIN RUN 8/10 - acc: 70.76% +- 1.36% - auc: 77.74% +- 2.08%



100%|██████████| 5000/5000 [00:58<00:00, 85.98it/s] 



GAIN RUN 9/10 - acc: 71.68% +- 3.63% - auc: 79.01% +- 4.52%



100%|██████████| 5000/5000 [00:59<00:00, 84.68it/s]



GAIN RUN 10/10 - acc: 73.96% +- 2.53% - auc: 80.32% +- 3.2%

GAIN GLOBAL - acc: 72.2% +- 0.86% - auc: 78.67% +- 0.81%


## Run MICE

In [7]:
imputer = IterativeImputer()
imputed = imputer.fit_transform(data_missing_nans)
## All runs would be the same since deterministic method
scores = utils.get_scores(imputed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MICE - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
mice_accs = acc
mice_aucs = auc

MICE - acc: 73.82% +- 0.0% - auc: 80.8% +- 0.0%


## Run MISSFOREST

In [8]:
miss_accs, miss_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputer = MissForest(n_jobs=-1, random_state=i)
    imputed = imputer.fit_transform(data_missing_nans)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nMISSFOREST RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    miss_accs.append(acc.mean())
    miss_aucs.append(auc.mean())
print(f'\nMISSFOREST GLOBAL - acc: {round(np.array(miss_accs).mean() * 100, 2)}% +- {round(np.array(miss_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(miss_aucs).mean() * 100, 2)}% +- {round(np.array(miss_aucs).std() * 100, 2)}%')

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5

MISSFOREST RUN 1/10 - acc: 71.96% +- 2.07% - auc: 79.34% +- 1.02%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3

MISSFOREST RUN 2/10 - acc: 71.98% +- 2.81% - auc: 79.38% +- 4.54%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5

MISSFOREST RUN 3/10 - acc: 71.84% +- 2.38% - auc: 79.63% +- 4.44%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4

MISSFOREST RUN 4/10 - acc: 71.23% +- 1.39% - auc: 78.24% +- 2.75%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4

MISSFOREST RUN 5/10 - acc: 73.03% +- 2.15% - auc: 79.11% +- 2.72%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5

MISSFOREST RUN 6/10 - acc: 72.56% +- 2.22% - auc: 78.15% +- 3.86%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5

MISSFOREST RUN 7/10 - acc: 72.56% +- 2.47% - auc: 79.73% +- 2.84%

Iteration: 0
Iteration: 1
Iter

## Run SOFTIMPUTE

In [9]:
soft_accs, soft_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputer = SoftImpute(random_state=i)
    imputer.fit(data_missing_nans)
    imputed = imputer.predict(data_missing_nans)
    imputed = np.where(missing_mask, data_missing, imputed)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'SOFTIMPUTE RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%')
    soft_accs.append(acc.mean())
    soft_aucs.append(auc.mean())
print(f'\nSOFTIMPUTE GLOBAL - acc: {round(np.array(soft_accs).mean() * 100, 2)}% +- {round(np.array(soft_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(soft_aucs).mean() * 100, 2)}% +- {round(np.array(soft_aucs).std() * 100, 2)}%')

SOFTIMPUTE RUN 1/10 - acc: 71.45% +- 3.64% - auc: 76.73% +- 3.44%
SOFTIMPUTE RUN 2/10 - acc: 71.73% +- 3.84% - auc: 78.25% +- 3.96%
SOFTIMPUTE RUN 3/10 - acc: 72.15% +- 3.1% - auc: 77.29% +- 2.72%
SOFTIMPUTE RUN 4/10 - acc: 72.46% +- 2.73% - auc: 78.17% +- 2.89%
SOFTIMPUTE RUN 5/10 - acc: 73.03% +- 2.77% - auc: 79.24% +- 2.33%
SOFTIMPUTE RUN 6/10 - acc: 71.94% +- 3.39% - auc: 77.54% +- 2.62%
SOFTIMPUTE RUN 7/10 - acc: 72.51% +- 2.8% - auc: 79.01% +- 3.1%
SOFTIMPUTE RUN 8/10 - acc: 73.74% +- 3.59% - auc: 78.73% +- 3.56%
SOFTIMPUTE RUN 9/10 - acc: 71.58% +- 2.23% - auc: 79.01% +- 1.8%
SOFTIMPUTE RUN 10/10 - acc: 71.52% +- 2.67% - auc: 79.3% +- 1.34%

SOFTIMPUTE GLOBAL - acc: 72.21% +- 0.7% - auc: 78.33% +- 0.85%


## Run SINKHORN

In [10]:
sink_accs, sink_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputer = OTimputer(niter=500)
    imputed = imputer.fit_transform(data_missing_nans).cpu().detach().numpy()
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nSINKHORN RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    sink_accs.append(acc.mean())
    sink_aucs.append(auc.mean())
print(f'\nSINKHORN GLOBAL - acc: {round(np.array(sink_accs).mean() * 100, 2)}% +- {round(np.array(sink_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(sink_aucs).mean() * 100, 2)}% +- {round(np.array(sink_aucs).std() * 100, 2)}%')

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 4.5090

SINKHORN RUN 1/10 - acc: 73.0% +- 3.45% - auc: 80.46% +- 5.69%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 4.5691

SINKHORN RUN 2/10 - acc: 72.34% +- 2.46% - auc: 80.02% +- 3.17%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 4.5750

SINKHORN RUN 3/10 - acc: 72.17% +- 2.91% - auc: 78.19% +- 3.85%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 4.5790

SINKHORN RUN 4/10 - acc: 71.85% +- 5.03% - auc: 77.91% +- 6.66%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 4.6528

SINKHORN RUN 5/10 - acc: 70.65% +- 3.07% - auc: 77.34% +- 4.8%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 4.7223

SINKHORN RUN 6/10 - acc: 72.87% +- 1.8% - auc: 80.23% +- 3.46%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 4.8059

SINKHORN RUN 7/10 - acc: 72.12% +- 2.92% - auc: 77.32% +- 4.97%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 4.7106

SINKHORN RUN 8/10 - acc: 72.77% +- 2.36% - auc: 78.3

## Run MIDA

In [11]:
mida_accs, mida_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputed = mida(data_missing_nans, num_layers=2, num_epochs=1500)
    imputed = np.where(missing_mask, data_missing, imputed)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nMIDA RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    mida_accs.append(acc.mean())
    mida_aucs.append(auc.mean())
print(f'\nMIDA GLOBAL - acc: {round(np.array(mida_accs).mean() * 100, 2)}% +- {round(np.array(mida_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(mida_aucs).mean() * 100, 2)}% +- {round(np.array(mida_aucs).std() * 100, 2)}%')

Stop training at epoch: 1500/1500, return best output

MIDA RUN 1/10 - acc: 71.88% +- 3.08% - auc: 78.37% +- 2.71%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 2/10 - acc: 72.82% +- 2.49% - auc: 78.77% +- 3.3%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 3/10 - acc: 72.88% +- 2.52% - auc: 78.23% +- 5.01%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 4/10 - acc: 72.26% +- 3.2% - auc: 77.48% +- 3.81%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 5/10 - acc: 72.69% +- 2.12% - auc: 77.36% +- 2.16%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 6/10 - acc: 69.65% +- 4.8% - auc: 75.71% +- 2.78%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 7/10 - acc: 72.77% +- 3.56% - auc: 77.39% +- 4.32%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 8/10 - acc: 71.01% +- 3.58% - auc: 79.81% +- 2.81%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 9/10 - acc:

## Run T-tests

In [12]:
for model, metrics in {
        'MEDIAN': {'ACC': median_accs, 'AUC': median_aucs},
        'MEAN': {'ACC': mean_accs, 'AUC': mean_aucs},
        'KNN': {'ACC': knn_accs, 'AUC': knn_aucs},
        'GAIN': {'ACC': gain_accs, 'AUC': gain_aucs},
        'MICE': {'ACC': mice_accs, 'AUC': mice_aucs},
        'MISS': {'ACC': miss_accs, 'AUC': miss_aucs},
        'SOFT': {'ACC': soft_accs, 'AUC': soft_aucs},
        'SINK': {'ACC': sink_accs, 'AUC': sink_aucs},
        'MIDA': {'ACC': mida_accs, 'AUC': mida_aucs}}.items():
    for metric_name, metric in metrics.items():
        ours_metric = ours_accs if metric_name == 'ACC' else ours_aucs
        t, p = scipy.stats.ttest_ind(np.array(ours_metric), np.array(metric))
        if p <= .05:
            if t > 0:
                ## Our method is better
                print(f'Metric {metric_name} - OUR METHOD is significantly better than {model}')
            else:
                ## Theirs is better
                print(f'Metric {metric_name} - OUR METHOD is significantly worse than {model}')
        else:
            ## Else we are even
            print(f'Metric {metric_name} - OUR METHOD is even with {model}')

Metric ACC - OUR METHOD is significantly better than MEDIAN
Metric AUC - OUR METHOD is significantly better than MEDIAN
Metric ACC - OUR METHOD is even with MEAN
Metric AUC - OUR METHOD is even with MEAN
Metric ACC - OUR METHOD is significantly better than KNN
Metric AUC - OUR METHOD is significantly better than KNN
Metric ACC - OUR METHOD is significantly better than GAIN
Metric AUC - OUR METHOD is significantly better than GAIN
Metric ACC - OUR METHOD is significantly better than MICE
Metric AUC - OUR METHOD is significantly better than MICE
Metric ACC - OUR METHOD is significantly better than MISS
Metric AUC - OUR METHOD is significantly better than MISS
Metric ACC - OUR METHOD is significantly better than SOFT
Metric AUC - OUR METHOD is significantly better than SOFT
Metric ACC - OUR METHOD is significantly better than SINK
Metric AUC - OUR METHOD is significantly better than SINK
Metric ACC - OUR METHOD is significantly better than MIDA
Metric AUC - OUR METHOD is significantly bet